In [26]:
import numpy as np
import scipy as sp
import pandas as pd

dim = 4

sigma_z = np.array([[1, 0],[0, -1]])
sigma_x = np.array([[0, 1], [1, 0]])
sigma_y = np.array([[0, -1j], [1j, 0]])
identity = np.identity(2)
sigmas = [identity, sigma_x, sigma_y, sigma_z]
sigmas_4d = np.kron(sigmas, sigmas)

# epsilon=0.0000001
# oepsilon=1-epsilon

def random_hamiltonian_and_rho(beta):
    a=np.random.uniform(-1,1,size=(dim,dim))+1j*np.random.uniform(-1,1,size=(dim,dim))
    H = (a.conj().T + a)/2
    eigenval,eigenvecs=np.linalg.eig(H)
    eigenval=np.e**(-beta*eigenval)
    eigenval=np.diag(eigenval)
    s=eigenval.dot(eigenvecs.conj().T)
    rho=eigenvecs.dot(s)
    rho=rho/np.trace(rho)
    return H, rho

def pure(beta):
    H=np.random.uniform(-1,1,size=dim)
    rho=np.e**(-beta*H)
    H=np.diag(H)
    rho=np.diag(rho)
    rho/=np.trace(rho)
    return H,rho

def matrix_to_free_parameters(matr):
    free_parameters = np.real(np.array([np.trace(matr.dot( measurement)) for measurement in sigmas_4d]))
    return free_parameters


def free_parameters_to_matrix(free_parameters):
    matr = np.zeros((4,4))+0j*np.zeros((4,4))
    for i in range(dim**2):
        matr += free_parameters[i]*sigmas_4d[i]

    return matr/4

# This function is new and generates non-equilibrium states and calculates the temperature according to the paper
def nongibbs():
    a=np.random.uniform(-1,1,size=(dim,dim))+1j*np.random.uniform(-1,1,size=(dim,dim))
    H = (a.conj().T + a)/2
    a=np.random.uniform(-1,1,size=(dim,dim))+1j*np.random.uniform(-1,1,size=(dim,dim))
    rho=a.dot(a.conj().T)
    rho/=np.trace(rho)
    t=np.trace(H)
    h=np.sqrt(np.trace(H.dot(H)-t**2/dim))
    o1=(H-t/dim)/h
    beta=np.trace(o1.dot(np.log(rho)))/(-h)
    return H,rho,beta
print(nongibbs())
# a=np.random.uniform(-1,1,size=(dim,dim))+1j*np.random.uniform(-1,1,size=(dim,dim))
# H = (a.conj().T + a)/2
# print(H - free_parameters_to_matrix(matrix_to_free_parameters(H)))


# hamiltonian , rho = hermitian(3, 0.5)
# print(rho - sp.linalg.expm(-0.5*hamiltonian)/np.trace(sp.linalg.expm(-0.5*hamiltonian)))


# a = np.array([1,2,3])
# print(np.outer(a, a.conj()))



(array([[ 0.72977677+0.j        ,  0.20640347+0.01222034j,
         0.1282554 +0.11631798j, -0.51156968+0.53241966j],
       [ 0.20640347-0.01222034j,  0.38039274+0.j        ,
         0.23708315-0.47629055j,  0.44178932+0.60687713j],
       [ 0.1282554 -0.11631798j,  0.23708315+0.47629055j,
         0.55825138+0.j        , -0.5438632 +0.53629711j],
       [-0.51156968-0.53241966j,  0.44178932-0.60687713j,
        -0.5438632 -0.53629711j,  0.25750934+0.j        ]]), array([[ 0.21164893+0.j        , -0.02055132+0.06456486j,
        -0.01999685+0.11379764j, -0.03993351-0.03092397j],
       [-0.02055132-0.06456486j,  0.30666716+0.j        ,
         0.16507618+0.04953671j, -0.2768641 +0.11251626j],
       [-0.01999685-0.11379764j,  0.16507618-0.04953671j,
         0.15627917+0.j        , -0.12422175+0.09919618j],
       [-0.03993351+0.03092397j, -0.2768641 -0.11251626j,
        -0.12422175-0.09919618j,  0.32540474+0.j        ]]), (-11.810130689171071-0j))


In [5]:
numdata = 10**5




cols = []

for i in range(1, 17):
    cols.append('fp hamiltonian'+str(i))

for i in range(1, 17):
    cols.append('fp rho'+str(i))

cols.append('beta')

# H=np.zeros((numdata,dim,dim),dtype=np.complex_)
# rho=np.zeros((numdata,dim,dim),dtype=np.complex_)
beta = np.random.uniform(0, 1, numdata)

hamiltonian, rho = random_hamiltonian_and_rho(beta[0])

free_params_hamiltonian = matrix_to_free_parameters(hamiltonian)
free_params_rho = matrix_to_free_parameters(rho)

data = np.append( np.append(free_params_hamiltonian, free_params_rho), np.array(beta[0])).reshape(1, 33)


# print(len(data), data)

df = pd.DataFrame(data, columns=cols)
print(df)
for i in range(1,numdata//10):
    hamiltonian,rho=pure(beta[i])
    free_params_hamiltonian = matrix_to_free_parameters(hamiltonian)
    free_params_rho = matrix_to_free_parameters(rho)
    data = np.append( np.append(free_params_hamiltonian, free_params_rho), np.array(beta[i]))
    df.loc[len(df)] = data
for i in range(numdata//10,numdata):
    # print(i)
    hamiltonian, rho = random_hamiltonian_and_rho(beta[i])

    free_params_hamiltonian = matrix_to_free_parameters(hamiltonian)
    free_params_rho = matrix_to_free_parameters(rho)

    data = np.append( np.append(free_params_hamiltonian, free_params_rho), np.array(beta[i]))
    df.loc[len(df)] = data



df.to_csv('QT - dimension 4.csv')

# for i in range(numdata):
#     H[i],rho[i]=hermitian(dim,beta[i])
# np.savez_compressed('Hamiltonians',H)
# np.savez_compressed('density matrices',rho)

   fp hamiltonian1  fp hamiltonian2  fp hamiltonian3  fp hamiltonian4  \
0         1.815262         0.063787         -0.98291         1.123441   

   fp hamiltonian5  fp hamiltonian6  fp hamiltonian7  fp hamiltonian8  \
0        -0.661134         1.925564         0.329642         -0.55569   

   fp hamiltonian9  fp hamiltonian10  ...   fp rho8   fp rho9  fp rho10  \
0        -0.408958          1.564257  ...  0.051871  0.056905 -0.161987   

   fp rho11  fp rho12  fp rho13  fp rho14  fp rho15  fp rho16      beta  
0  0.152104  0.039256  0.073198  0.019943  0.003156 -0.050396  0.438644  

[1 rows x 33 columns]


In [42]:
cols = []

for i in range(1, 17):
    cols.append('fp hamiltonian'+str(i))

for i in range(1, 17):
    cols.append('fp rho'+str(i))

cols.append('beta')
numdata=2*10**4
hamiltonian, rho, betas = nongibbs()
free_params_hamiltonian = matrix_to_free_parameters(hamiltonian)
free_params_rho = matrix_to_free_parameters(rho)
print(betas,betas.real)
data = np.append( np.append(free_params_hamiltonian, free_params_rho), np.array(betas.real)).reshape(1, 33)
df = pd.DataFrame(data, columns=cols)
for i in range(1,numdata):
    hamiltonian, rho,betas = nongibbs()
    free_params_hamiltonian = matrix_to_free_parameters(hamiltonian)
    free_params_rho = matrix_to_free_parameters(rho)
    data = np.append( np.append(free_params_hamiltonian, free_params_rho), np.array(betas.real))
    df.loc[len(df)] = data
df.to_csv('nonthermal - dimension 4.csv')

(-1.384321226799688-0j) -1.384321226799688
